Actividad 1 

Explicacion del preprocesamiento hecho y/o fotos del preprocesamiento del csv

Actividad 2 

En este caso se usó <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html">KFold</a>. Recuerde que también existe <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html">train_test_split</a>.


In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

# Cargar los datos desde los archivos CSV
archivo_taller = 'Taller_2_Titulacion_DatosTaller.csv'
datos_taller = pd.read_csv(archivo_taller, encoding='latin-1', delimiter=';')

# Separar las características y la variable objetivo
X_taller = datos_taller.iloc[:, :-1]  
y_taller= datos_taller.iloc[:, -1]

# Eliminar la columna "Id" de ambos conjuntos de datos
X_taller = datos_taller.drop('Id', axis=1)

# Convertir variables categóricas en dummies
X_taller = pd.get_dummies(X_taller)

# Convertir las etiquetas categóricas en valores binarios
y_taller = y_taller.map({"chileno": 1, "extranjero": 0, "mujer": 1, "hombre": 0}).fillna(0).values.ravel()

# Reemplazar NaN por 0 en el conjunto de entrenamiento
X_taller = X_taller.fillna(0)

# Crear objeto KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Visualizar las primeras filas de los datos procesados
print(X_taller.head())
print(y_taller[:5])



   MAT_1SEM_PROM  FIS_1SEM_PROM  ING_1SEM_PROM  ACTF_1SEM_A  ACTF_1SEM_R  \
0           55.0            0.0           79.0          1.0          0.0   
1           56.0           57.0           88.0          0.0          0.0   
2           55.0           64.0           55.0          0.0          0.0   
3            0.0           56.0            0.0          0.0          0.0   
4           87.0           82.0           86.0          0.0          0.0   

   OTRANS_1SEM_A  OTRANS_1SEM_R  OTRANS_1SEM_PROM  ESP_1SEM_A  ESP_1SEM_R  \
0            2.0            0.0              67.0         2.0         0.0   
1            0.0            0.0               0.0         4.0         0.0   
2            0.0            0.0               0.0         4.0         0.0   
3            2.0            0.0              64.0         2.0         0.0   
4            0.0            0.0               0.0         3.0         1.0   

   ...  \
0  ...   
1  ...   
2  ...   
3  ...   
4  ...   

   descripcion_rama

Decidimos utilizar la validación cruzada porque nos permite evaluar cómo se comportará nuestro modelo en diferentes conjuntos de datos. Esto nos da una visión más robusta de su desempeño y nos ayuda a evitar sesgos que podrían surgir al dividir los datos en un solo conjunto de entrenamiento y prueba.

Antes de aplicar la validación cruzada, nos aseguramos de que nuestros datos estuvieran limpios y listos para usar. Eliminamos columnas innecesarias, convertimos las categorías en valores numéricos y tratamos los valores faltantes para garantizar la coherencia y la integridad de nuestros datos.

Actividad 3

En este caso debieran que ser 4 bloques 2 modelos y dos modificaciones de parametros para cada modelo 

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# Definir los modelos y configuraciones de hiperparámetros
models = [
    ('Random Forest', RandomForestClassifier(),
     {'n_estimators': [100, 200], 'max_depth': [None, 10, 20]}),
    ('Decision Tree', DecisionTreeClassifier(),
     {'max_depth': [None, 10, 20]})
]

# Ejecutar búsqueda del mejor clasificador para cada modelo
best_models = {}
for name, model, params in models:
    print(f'Buscando el mejor clasificador para {name}...')
    clf = GridSearchCV(model, params, cv=5, scoring='accuracy')
    clf.fit(X_taller, y_taller)
    best_models[name] = clf.best_estimator_
    print(f'Mejor clasificador encontrado para {name}: {clf.best_params_}')

# Seleccionar el modelo con mejor rendimiento
best_model_name = max(best_models, key=lambda k: clf.cv_results_['mean_test_score'][clf.best_index_])
best_model = best_models[best_model_name]
best_score = clf.best_score_
print(f'\nMejor modelo encontrado: {best_model_name}')
print(f'Puntaje del mejor modelo: {best_score:.4f}')


# Evaluar el mejor modelo
best_model.fit(X_taller, y_taller)
print(f'Precisión en el conjunto de entrenamiento: {best_model.score(X_taller, y_taller):.4f}')


Buscando el mejor clasificador para Random Forest...
Mejor clasificador encontrado para Random Forest: {'max_depth': None, 'n_estimators': 100}
Buscando el mejor clasificador para Decision Tree...
Mejor clasificador encontrado para Decision Tree: {'max_depth': None}

Mejor modelo encontrado: Random Forest
Puntaje del mejor modelo: 1.0000
Precisión en el conjunto de entrenamiento: 1.0000


Optamos por probar dos modelos diferentes, Random Forest y Árbol de Decisión. Seleccionamos Random Forest debido a su capacidad para manejar conjuntos de datos grandes y complejos, mientras que elegimos Árbol de Decisión por su simplicidad y facilidad de interpretación.

Creemos que Random Forest es como un equipo diverso que trabaja en colaboración para tomar decisiones, mientras que el Árbol de Decisión es más como un individuo que sigue una serie de reglas predefinidas. Queríamos explorar ambas opciones para obtener una comprensión completa de cómo se comportarán nuestros modelos en nuestro problema específico.

Actividad 4

Aplicacion del modelo a nuevo conjunto

In [40]:
import pandas as pd

# Cargar los datos de evaluación
datos_evaluacion = pd.read_csv('Taller_2_Titulacion_Evaluación.csv', encoding='latin-1', delimiter=';')

# Eliminar la columna "Id" si es necesario
X_evaluacion = datos_evaluacion.drop('Id', axis=1)

# Convertir variables categóricas en dummies si es necesario
X_evaluacion = pd.get_dummies(X_evaluacion)

# Hacer predicciones con el modelo cargado
predictions = model.predict(X_evaluacion)

# Convertir las predicciones a etiquetas legibles
predicted_labels = np.where(predictions[:, 1], 'SÍ', 'NO')  # Si la segunda columna (índice 1) es True, entonces 'SÍ', de lo contrario 'NO'

# Crear un DataFrame con las columnas Id y Prediccion
resultados = pd.DataFrame({'Id': datos_evaluacion['Id'], 'Prediccion': predicted_labels})

# Guardar el DataFrame como un archivo de texto separado por comas
resultados.to_csv('clasificacion_titulo.txt', index=False, sep=',', header=True)


NotFittedError: This DecisionTreeClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.